# 3.1. Бинарная кросс-энтропия

В сегодняшней домашней работе вам предлагается самостоятельно написать и обучить модель логистической регрессии для решения задачи бинарной классификации.

Реализация логистической регрессии с имеющимся заранее вектором весов \vec{w} - задача достаточно простая. Вся загадка кроется в вопросе о том, как подобрать оптимальный вектор весов \vec{w}. Прежде чем перейти к вопросу о подборе этого вектора необходимо определиться с тем, в каком смысле мы понимаем его оптимальность. То есть что является критерием его оптимальности? Как сказать, что один вектор \vec{w_1} более оптимален, чем другой \vec{w}?

Для определения того, насколько хорош некоторый вектор параметров для решения нашей конкретной задачи, вводится понятие функции потерь (Loss-функции). Значение этой функции обратно пропорционально качеству решения нашей задачи. То есть оптимальным вектором параметров мы теперь можем назвать тот, при котором значение Loss-функции будет наименьшим из всех возможных. Таким образом мы сводим решение задачи машинного обучения к решению задачи оптимизации.

Для задачи бинарной классификации очень популярна функция потерь, называемая бинарной кросс-энтропией. Эта функция пришла в машинное обучение из теории информации. Она имеет следующий вид:

$$H(p, y) = - (y \cdot ln(p) +(1 - y) \cdot ln(1-p))$$


Где $y$ - верный ответ (0 или 1), а $p$ - наше предположение, выраженное в оцененной степени принадлежности очередного объекта $x$ классу 1.

Для обучения модели мы решим воспользоваться алгоритмом градиентного спуска.

В связи с этим, нам необходимо предварительно посчитать все производные функции H по параметрам модели \vec{w_1}...\vec{w_n}.

Выведите формулы этих производных и посчитайте производную функции H в точке

$$w_0 = 0$$
$$w_1 = 2$$
$$w_2 = -1$$

По каждому из этих параметров, если рассматриваемый объект x, имеющий координаты векторного представления 
$x_1=1$ и $x_2=$ принадлежит классу 1.

В ответ в контесте внесите значение произведения $\frac{∂H}{w_0}$$\frac{∂H}{w_1}$$\frac{∂H}{w_2}$

In [1]:
import numpy as np

y = 1
w = np.array([0, 2, -1])
x = np.array([1, 1, 2])
o = np.array([1, 1, 1])


def sigmoid(x: float) -> float:
    return 1 / (1 + np.math.exp(-x))


p = sigmoid(np.dot(x, w))


def f(x: float) -> float:
    return - (1 - p) * x


np.product([f(i) for i in x])

-0.25

# 3.2 Градиентный спуск

В этом задании нам предстоит реализовать классический алгоритм градиентного спуска для обучения модели логистической регрессии.

Алгоритм выполнения этого задания следующий:

* На основе посчитанных в первом задании частных производных, напишем функцию подсчета градиента бинарной кросс-энтропии по параметрам модели

* Напишем функцию обновления весов по посчитанным градиентам 

* Напишем функцию тренировки модели

Замечание:
Тренировка модели проводится в несколько циклов, в рамках каждого из которых мы обновим веса модели, основываясь на предсказании для **каждого** объекта из датасета. Такие циклы называются *эпохами*. То есть одна эпоха - это набор обновлений весов, реализованный согласно посчитанным для каждого объекта из датасета ошибкам модели.

Вам необходимо реализовать обучение модели в несколько эпох. Их количество задается параметром функции. В рамках каждой эпохи необходимо пройти циклом по всем объектам обучающей выборки и обновить веса модели.

# Замечание:

В случае, если у Вас возникли сложности с выполнением первого задания и, как следствие, у Вас не выходит сделать это, мы рекомендуем подробно ознакомиться с главой **Производные $\frac{\partial H}{\partial \omega_i}$** нашей [лекции](https://colab.research.google.com/drive/1xjX_YnXcRr8HSiYLByMHxEIAADqs7QES?usp=sharing)

In [2]:
import numpy as np

# Функция - Сигмоида
def sigmoid(x: float) -> float:
    return 1 / (1 + np.exp(-x))


# Функция предсказания модели
def predict(features: np.array, weights: np.array) -> float:
    x = np.dot(features, weights)
    return sigmoid(x)


# Функция подсчета градиента
def gradient(y_true: int, y_pred: float, x: np.array) -> np.array:
    """
    y_true - истинное значение ответа для объекта x
    y_pred - значение степени принадлежности объекта x классу 1, предсказанное нашей моделью
    x - вектор признакового описания данного объекта

    На выходе ожидается получить вектор частных производных H по параметрам модели, предсказавшей значение y_pred
    Обратите внимание, что размерность этого градиента должна получиться на единицу больше размерности x засчет своободного коэффициента a0
    """
    grad = x * (y_pred - y_true)
    return grad


# Функция обновления весов
def update(alpha: np.array, gradient: np.array, lr: float):
    """
    alpha: текущее приближения вектора параметров модели
    gradient: посчитанный градиент по параметрам модели
    lr: learning rate, множитель перед градиентом в формуле обновления параметров
    """
    alpha_new = alpha - lr * gradient
    return alpha_new

#функция тренировки модели
def train(alpha0: np.array, x_train: np.array, y_train: np.array, lr: float, num_epoch: int):
    """
    alpha0 - начальное приближение параметров модели
    x_train - матрица объект-признак обучающей выборки
    y_train - верные ответы для обучающей выборки
    lr - learning rate, множитель перед градиентом в формуле обновления параметров
    num_epoch - количество эпох обучения, то есть полных 'проходов' через весь датасет
    """
    alpha = alpha0.copy()
    for epo in range(num_epoch):
        for i, x in enumerate(x_train):
            x = np.hstack([x, 1])
            prediction = predict(x, alpha)
            grad = gradient(y_train[i], prediction, x)
            alpha = update(alpha, grad, lr)
    return alpha

In [83]:
import random
import numpy as np

In [96]:
n_features = 10

alpha = np.zeros(n_features)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def gradient_boosting(X, y, lr, alpha=np.zeros(n_features), n_iter=100):
    alpha = alpha.copy()
    for i in range(n_iter):
        grad = 2 * (X @ alpha - y) @ X
        alpha -= lr * grad
    return alpha


def stohastic_gradient_boosting(X, y, lr, alpha=np.zeros(n_features), n_iter=100):
    alpha = alpha.copy()
    for i in range(n_iter):
        x = X[np.random.randint(0, 442, size=10)]
        grad = 2 * (x @ alpha - y) @ X
        alpha -= lr * grad
    return alpha

array([367,  34,   6, 303,  11,  53, 337, 165, 363, 236])

In [97]:
2 * (X[0] @ alpha - y[0]) * X[0]

array([-11.49892374, -15.30539586, -18.63225437,  -6.60545121,
        13.35549652,  10.51587038,  13.10705539,   0.78286312,
        -6.0123431 ,   5.3291298 ])

In [111]:
gradient_boosting(X, y, 0.01, n_iter=3000)

array([  -7.90862466, -237.46573876,  525.16977942,  322.33448712,
       -335.67716673,  114.28444259, -102.85626314,  119.24542186,
        581.06145778,   69.29580816])

In [106]:
stohastic_gradient_boosting(X, y, 0.01, n_iter=50000)

array([-2.52055262e-01, -2.23142385e+02,  4.78083673e+02,  3.14840524e+02,
       -3.98494520e+01, -1.23654339e+02, -1.90395119e+02,  1.20275390e+02,
        4.24323033e+02,  9.56434774e+01])

# L2-регуляризация

Регуляризация - это один из способов борьбы с переобучением моделей. Делается это в основном при помощи добавления к функции потерь некоторых слагаемых. Существуют стандартные методы регуляризации. Один из таких методов - это так называемая L2-регуляризация. Состоит идея L2-регуляризации в следующем:

Мы добавляем к функции потерь специальное слагаемое, равное половине квадрата 2-нормы (длины вектора) весов модели, умноженного на некоторый коэффициент. То есть наша новая лосс функция выглядит так:

$$\hat{H} = H + \frac{β}{2}|\vec{\omega}|^2$$

Градиенты такой функции тоже нехитрым образом преобразуются:
$$\frac{∂ \hat{H}}{∂ \omega} = \frac{∂H}{∂ \omega} + β\omega$$
Поскольку $|\vec{\omega}|^2 = \omega_0^2 + \omega_1^2 + ... + \omega_n^2$

Реализуйте класс LogisticRegression для решения задачи бинарной классификации с L2-регуляризацией.

Напоминание:

* Функция .fit(x, y) производит обучение модели. В рамках этой функции необходимо реализовать подбор оптимальных параметров модели/сконфигурировать модель для дальнейшего использования на основе данной тренировочной выборки, где x - это матрица признакового описания выборки, а y - вектор ответов

* Функция .predict(x) осуществляет предсказание для каждого из объектов, чьи векторные описания представлены строками матрицы x. Выполняется строго после .fit(). Ради безопасности можно даже реализовать механизм отказа в виде выбрасывания специальной ошибки UnfittedError в случае попытки вызова функции .predict() до вызова функции .fit()

# Замечание

По большому счету Вам нужно внести соответствующие изменения в класс LogisticRegression, который Вы реализовали в предыдущем задании. В качестве шаблона кода можно также взять код из предыдущего задания. Главное требование - реализация функций `.fit()`,  `.predict()` и `predict_proba()`

In [3]:
import numpy as np


class UnfittedError(Exception):
    pass


class LogisticRegression(object):
    
    def __init__(self):
        self.alpha = None

    @staticmethod
    def _sigmoid(x: float) -> float:
        return 1 / (1 + np.exp(-x))
    
    @staticmethod
    def _add_constant_column(X: np.array) -> np.array:
        if len(X.shape) > 1:
            return np.hstack([X, np.ones((X.shape[0], 1))])
        return np.hstack([X, 1])
    
    @staticmethod
    def _gradient(y_true: int, y_pred: float, x: np.array) -> np.array:
        grad = x * (y_pred - y_true)
        return grad
    
    @staticmethod
    def _update(alpha: np.array, gradient: np.array, lr: float) -> np.array:
        alpha_new = alpha - lr * gradient
        return alpha_new
    
    def fit(self, x_train: np.array, y_train: np.array, lr: float, betta: float, num_epoch: int) -> None:
        alpha = np.ones(x_train.shape[1] + 1)
        for epo in range(num_epoch):
            for i, x in enumerate(x_train):
                x = self._add_constant_column(x)
                prediction = self._sigmoid(np.dot(x, alpha))
                grad = self._gradient(y_train[i], prediction, x) + betta * alpha
                alpha = self._update(alpha, grad, lr)
        self.alpha = alpha
    
    def predict(self, X):
        if self.alpha is None:
            raise UnfittedError
        preds = self._sigmoid(np.dot(self._add_constant_column(X), self.alpha))
        return preds